In [1]:
import torch

from models import inception_resnet_v1
from models.utils.dataset_generator import generate_dataset
from models.utils.dataset import TripletFaceDataset
from torchvision.transforms import v2
from torch.utils.data import DataLoader
from models.utils import embed_train

In [2]:
dataset_df = generate_dataset("E:\\105_classes_pins_dataset")

In [3]:
transform = v2.Compose([
    v2.RandomAutocontrast(p=0.13),
    v2.RandomAdjustSharpness(2, 0.12)
])

dataset = TripletFaceDataset(
    triplets_dataframe=dataset_df,
    image_size=128,
    transform=transform
)

dataloader = DataLoader(
    dataset=dataset,
    batch_size=2,
    num_workers=1
) 

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = inception_resnet_v1.InceptionResnetV1(
    pretrained='vggface2',
    device=device
)

In [ ]:
EPOCHS = 10
for i in range(EPOCHS):
    model, loss = embed_train.train_one_epoch(
                       model=model,
                       optimizer='sgd',
                       lr=0.001,
                       momentum=0.9,
                       margin=1.3,
                       train_loader=dataloader,
                       log_interval=5,
                       device=device
                   )
    print(loss)

	Batch: 0, Loss: 0.1218
	Batch: 5, Loss: 0.0430
	Batch: 10, Loss: 0.0000
	Batch: 15, Loss: 0.0000
	Batch: 20, Loss: 0.0000
	Batch: 25, Loss: 0.0000
	Batch: 30, Loss: 0.0000
	Batch: 35, Loss: 0.0000
	Batch: 40, Loss: 0.0000
	Batch: 45, Loss: 0.0000
